In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, MaxPooling1D, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')
import re
seed = 1234
np.random.seed(seed)

ModuleNotFoundError: No module named 'numpy'

In [6]:
df = pd.read_csv('train.csv')
is_toxic = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] == 0).all(axis=1)
not_toxic = [d for d in range(0, len(df)) if is_toxic[d]]
toxic = [d for d in range(0, len(df)) if not is_toxic[d]]
X_train, X_test, Y_train, Y_test = train_test_split(df["comment_text"],is_toxic)
print(len(df))
print(len(X_train))

159571
119678


In [7]:
swords = stopwords.words("english")
swords.append("im")


def modify_input(tokens):
    new_list = []
    for token in tokens:
        if token not in swords:
            word = re.sub(r'[^A-Za-z0-9\s]', '', token)
            if(len(word) > 0):
                new_list.append(word)
    return new_list

In [8]:
tokens = [word_tokenize(sen.lower().strip()) for sen in X_train]
clean_tokens = []
for token_list in tokens:
    clean_tokens.append(modify_input(token_list))

In [9]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
df = pd.read_csv('train.csv')
is_toxic = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] == 0).all(axis=1)
not_toxic = [d for d in range(0, len(df)) if is_toxic[d]]
toxic = [d for d in range(0, len(df)) if not is_toxic[d]]
X_train, X_test, Y_train, Y_test = train_test_split(df["comment_text"],is_toxic)
# Tokenization using Keras Tokenizer
tokenizer = Tokenizer(num_words=20000)  # Limit vocabulary size
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# Padding sequences to ensure same length
max_seq_len = 100
train_padded = pad_sequences(train_sequences, maxlen=max_seq_len)
test_padded = pad_sequences(test_sequences, maxlen=max_seq_len)
print(train_padded.shape)

(119678, 100)


In [11]:
# model = Sequential([
#   Embedding(39450, 16),
#   Dropout(0.2),
#   GlobalAveragePooling1D(),
#   Dropout(0.2),
#   Dense(1, activation='sigmoid')])


model = Sequential()
model.add(Embedding(20000, 100))
#model.add(Dense(128, activation = 'relu'))
model.add(Conv1D(filters = 256, kernel_size = 3, activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # For binary classification
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                              tf.keras.metrics.Precision(),
                              tf.keras.metrics.Recall(),
                              tf.metrics.F1Score()])
model.fit(train_padded, Y_train, validation_data = (test_padded, Y_test), batch_size = 32, epochs = 2)

Epoch 1/2


ValueError: FBetaScore expects 2D inputs with shape (batch_size, output_dim). Received input shapes: y_pred.shape=(None, 1) and y_true.shape=(None,).

In [45]:
accuracy = [model.evaluate(test_padded, Y_test, verbose=0)]
print(f"Accuracy: {accuracy}")

Accuracy: [[0.11291726678609848, 0.9607951045036316, 0.9673316478729248, 0.9897517561912537, 0.946079432964325]]


In [8]:
testl = pd.read_csv('test.csv')
testv = pd.read_csv('test_labels.csv')

In [92]:
tokenizer.fit_on_texts(testl["comment_text"])
tl = tokenizer.texts_to_sequences(testl["comment_text"])
max_seq_len = 100
tlp = pad_sequences(tl, maxlen=max_seq_len)

0         False
1         False
2         False
3         False
4         False
          ...  
153159    False
153160    False
153161    False
153162    False
153163    False
Length: 153164, dtype: bool


In [99]:
is_toxic = (testv[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] == 0).all(axis=1)
print(is_toxic)
toxic = [d for d in range(0, len(df)) if not is_toxic[d]]
print(testl["comment_text"][3])

0         False
1         False
2         False
3         False
4         False
          ...  
153159    False
153160    False
153161    False
153162    False
153163    False
Length: 153164, dtype: bool
:If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message.


In [91]:
loss, accuracy = model.evaluate(tlp,is_toxic, verbose=0)
print(f"Accuracy: {accuracy}")

Accuracy: 0.4027186632156372
